# Getting Started with Google Cloud

Manav Agarwal <manav@me.com>, Shanaka Desosya <shanaka@computer.org>, Suresh Seeram <suresh.b.seeram@gmail.com> <br>
Freel free to contact for questions or support.

## Overview

Google Cloud is Google's cloud server offering, much like Amazon's EC2. Recently Google started offering instances with GPU support (NVIDIA K80). This is great for those of you who want to run GPU accelerated machine learning algorithms, but don't have the hardware.

Google is offering a $300 credit for new signups which should get you roughly 300 hours of usage. As long as you remember to stop your instance when you are done training your models, this should get you a good amount of use. 

### Step 1: Registration & Billing

To get started, go to https://console.cloud.google.com/. Create and name your initial project. 

<img src='http://www.manavagarwal.com/wp-content/uploads/2017/05/g-ss1.png'>

Google GPU instances are disabled for new accounts. To enable them, you must first upgrade your account by entering in billing information. Don't worry because you will only get charged once you surpass your free limit. Once again, remember to stop your instances when you are done. Next you upgrade your account to a paid account by clicking on the free trial status on top.  

<img src='http://www.manavagarwal.com/wp-content/uploads/2017/05/g-ss2.png'>

### Step 2: Request Quota Increase

To enable GPUs on your account you have to submit a request to Google which is usually automatically processed. First, click on Compute Engine on the left tab. Then navigate to Quotas and click "Request Quota Increase"

<img src='http://www.manavagarwal.com/wp-content/uploads/2017/05/g-ss3.png'>

Fill out the form requesting one or two GPU dies. For region, I selected US-WEST1 since that is closest to me and has GPUs available. Offhand, I know they are also available in the US-EAST region. You only need to enter in your name, email, phone and the amount of GPUs. 

After filling out the form you should hear back from Google via email immediately granting your request. 

### Step 3: Start an Instance

Navigate back to the Compute Engine tab and click on VM Instances -> Create Instance. You will have to click on the "customize" tab to access GPU options.

<img src='http://www.manavagarwal.com/wp-content/uploads/2017/05/g-ss4.png'>

I went with the following configuration (your own needs may vary):<br>
Cores: 4<br>
Memory: 15gb<br>
**GPU: 1 NVIDIA K80**<br>
<br>
Boot Disk: Ubuntu 16.04 LTS 250GB.<br>
Allow All API Access<br>
Allow HTTP/HTTPS Traffic.<br>

Click create and your instance will be ready to go.

### Step 4: Setup SSH key

You can SSH to your instance directly from the Google console, but you may want to setup a SSH key to make it easier to transfer files or run something like Jupyter notebook.

Please refer to: https://cloud.google.com/compute/docs/instances/adding-removing-ssh-keys. Your instructions will vary by OS and configuration (or you probably already have one setup). The basic instructions are as follows:

In [ ]:
~$ ssh-keygen -t rsa -f ~/.ssh/my-google-ssh-key -C [USERNAME/GoogleID]
~$ chmod 400 ~/.ssh/my-google-ssh-key
~$ cat ~/.ssh/my-google-ssh-key.pub

Once you have your public key generated, navigate to the Metadata tab and paste it under "SSH Keys". Your keys will be transfered to the server and ready for use. Go back to the VM Instances tab and take note of your "External IP" You can then ssh into your instance with the following:

In [ ]:
~$ ssh -i ~/.ssh/my-google-ssh-key [USERNAME]@[EXTERNAL_IP]

### Step 5: Install Cuda

SSH into your instance and do the following:

In [ ]:
~$ curl -O http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
~$ sudo dpkg -i cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
~$ sudo apt-get update
~$ rm cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
~$ sudo apt-get install cuda -y

To confirm, run nvidia-smi

In [1]:
~$ nvidia-smi

Sun May  7 22:15:14 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.51                 Driver Version: 375.51                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:04.0     Off |                    0 |
| N/A   47C    P0    62W / 149W |  10941MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Export environmental variables.

In [ ]:
~$ echo 'export CUDA_HOME=/usr/local/cuda' >> ~/.bashrc
~$ echo 'export PATH=$PATH:$CUDA_HOME/bin' >> ~/.bashrc
~$ echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CUDA_HOME/lib64' >> ~/.bashrc
~$ source ~/.bashrc

### Step 6: Install cuDNN

cuDNN is available with free license from Nvidia. You must register at https://developer.nvidia.com/cudnn to download cuDNN. Once you register, make sure to download cudnn-8.0-linux-x64-v5.1.tgz. cuDNN 8.0 for Linux v5.1 (only v5.1 will work with Tensorflow's default installation)

Next copy the file to your instance. You can do this via scp or sftp with the ssh key you created earlier.

In [ ]:
$ scp -i .ssh/my_ssh_key cudnn-8.0-linux-x64-v5.1.tgz <instance-ip>:

Once you are back SSHed into your instance, do the following:

In [ ]:
~$ cd
~$ tar xzvf cudnn-8.0-linux-x64-v5.1.tgz
~$ sudo cp cuda/lib64/* /usr/local/cuda/lib64/
~$ sudo cp cuda/include/cudnn.h /usr/local/cuda/include/
~$ rm -rf ~/cuda
~$ rm cudnn-8.0-linux-x64-v5.1.tgz

After this cuda/cudnn should be up and running. 

### Step 7: Install Anaconda and Deep Learning Frameworks

Anaconda is a good way of installing Jupyter Notebook as well as a lot of basic python libraries. We're using the Python 2.7 version of Anaconda. 

In [ ]:
~$ curl -O https://repo.continuum.io/archive/Anaconda2-4.3.1-Linux-x86_64.sh
~$ bash Anaconda2-4.3.1-Linux-x86_64.sh

Next make sure you have pip installed. We're using and installing for python 2.7 in this instance.

In [ ]:
~$ sudo apt-get install python-pip

Install tensorflow-gpu and keras (or other deep learning frameworks). Also install opencv

In [ ]:
~$ pip install tensorflow-gpu
~$ pip install keras
~$ conda install opencv

### Step 8: (Optional) Configure and run Jupyter Notebooks

First cd to your home directory. We make the notebooks directory (don't cd into it yet) and generate the config file.

In [ ]:
~$ mkdir notebooks
~$ jupyter notebook --generate-config

Then use vim/nano to edit your configuration file 

In [ ]:
~$ vim ~/.jupyter/jupyter_notebook_config.py

You have a few options here. 
Personally I like to set up a SSH tunnel. For that configuration enter the following at top of the file: 

*Option 1: SSH Tunnel*

In [ ]:
c = get_config()  # get the config object
c.IPKernelApp.pylab = 'inline'  # in-line figure when using Matplotlib
c.NotebookApp.open_browser = False  # do not open a browser window by default when using notebooks
c.NotebookApp.token = '' # No token. But, please, please always use jupyter over ssh tunnel

In [ ]:
Then on your local machine you do the following

In [ ]:
~$ ssh -i .ssh/my_google_ssh_key -L 8899:localhost:8888 username@<instance-ip>

In [ ]:
Jupyter should be accessible then on your local machine at https://localhost:8899

*Option 2: HTTP over web*

First generate a password. Open Jupyter Notebook or a python shell locally and enter: 

In [ ]:
from notebook.auth import passwd; passwd()

This will generate a sha1 hash after you enter a password. You can use some other type of hash just update appropriately.

In [ ]:
~$ vim ~/.jupyter/jupyter_notebook_config.py

Then edit the same file from above with the following contents, inserting your key.

In [ ]:
~$ c.NotebookApp.allow_origin = '*'
~$ c.NotebookApp.ip = '*'
~$ c.NotebookApp.open_browser = False
~$ c.NotebookApp.password = u'sha1:yourkeyhere'

Run your notebook by going to <instance-ip>:8888 on your local browser. Enter your password and it should work. As a demo do the following.

In [2]:
import keras

Using TensorFlow backend.


In your log it should show your GPU info confirming its using the GPU.

<img src='http://www.manavagarwal.com/wp-content/uploads/2017/05/g-ss5.png'>